In [2]:
import tensorflow as tf
import os 
import numpy as np
import pandas as pd
from model import CNN_Model

In [3]:
class Policy_Network(CNN_Model):
    def __init__(self,param_dict={},restore_params=False,pickle_file_path=""):
        CNN_Model.__init__(self,param_dict={},restore_params=False,pickle_file_path="")
        
    def form_loss(self,logits,targets):
        entropies=self.params.loss_fn(labels=targets,logits=logits)
        return entropies
        
    def Build_model(self):
        self.build_model_till_logits()
        with tf.variable_scope(self.params.name_scope):
            
#             self.Advantage=self.form_placeholder((None,1),tf.float32)
            self.probs_all_actions=tf.nn.softmax(self.logits)#converting to probs
            log_likelihood_all_actions=tf.log(self.probs_all_actions)
        
            #which action to take
            self.desired_action=tf.multinomial(log_likelihood_all_actions,num_samples=1)

#           we will be using sparse softmax cross entropy function which will give us neg ;p likelihoods for yje selected action
            self.neg_log_likelihood=self.form_loss(self.logits,self.desired_action[:][0])
           #computing gradients 
            optimizer=self.params.optimizer_fn(learning_rate=self.lr_placeholder)
            self.grads_and_vars=optimizer.compute_gradients(loss=self.neg_log_likelihood,var_list=self.model_variables)
            
            
#             #for easy manipulation of grads
            self.grads=np.array([grad for grad,var_name in self.grads_and_vars])
            
#             #placeholder for all gradients (list of placeholder and var name tuples)
            self.grad_placeholders=[self.form_placeholder(grad.get_shape(),tf.float32) for grad,_ in self.grads_and_vars]
            self.grads_and_vars_feed=[(self.grad_placeholders[i],self.grads_and_vars[i][1]) for i in np.arange(len(self.grads_and_vars)) ]
            #linking placeholder and var name
            
#             #taking grad step based on the values of grads received 
            self.train_op=optimizer.apply_gradients(grads_and_vars=self.grads_and_vars_feed,global_step=self.step_no)
            
            #pandas implementation
#             grads_and_vars_tuples=optimizer.compute_gradients(loss=self.neg_log_likelihood,var_list=self.model_variables)
#             self.grads_and_vars=pd.DataFrame(grads_and_vars_tuples,columns=["grad","var_name"])
            
#             self.grads_and_vars_feed=self.grads_and_vars
#             sef
            self.initializer=tf.global_variables_initializer()